In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [2]:
# MODIFY THIS LINE!!!
# Step 1: Save raw COD files from all campuses into separate directory to be referenced below
# Set path of directory with saved COD files and define all data file names from all campuses
# Set string aid_year long description based on the first DL filename
path =r'C:/Users/mjc341/Desktop/DirectLoans/08.13 Update'
path_final =r'C:/Users/mjc341/Desktop/DirectLoans/FinalVersions'
filenames = glob.glob(path + "/dsyh19op.*")

In [3]:
# Read data from individual COD DL csv files from all campuses
dl_csv_files = []
for filename in filenames:
    dl_csv_files.append(pd.read_csv(filename, skiprows=5, skipfooter=1, engine='python', index_col=False))

In [4]:
aid_year = '20' + str((int(filename[-8:-6])-1)) + '-20' + filename[-8:-6]
date = path[-12:-7]

In [5]:
# Aid Year string for DL Reconciliation
aid_year

'2018-2019'

In [6]:
date

'08.13'

In [7]:
# Combine data from all campuses into one DataFrame
dl_combined = pd.concat(dl_csv_files, ignore_index=True)

In [8]:
# Select records from COD DL combined file with Total Net Amt > 0
dl_gt_0 = dl_combined[dl_combined['TOTAL NET AMT']>0]

In [9]:
# Write COD DL file to Excel with all campuses combined
dl_gt_0.to_excel(path + "/dl_combined_" + filename[-8:-6] + ".xlsx", sheet_name='COD All Campuses', index=False)

In [10]:
# MODIFY THIS LINE TEMPORARILY UNTIL LOGIC IS CODED
# Open CaneLink DL Disbursement file regardless of the suffix added to filename by CaneLink Query Manager
canelink_dl_disb = pd.read_excel(path + "/AG_ALL_DISB_BY_YEAR_ALL_LOAN_8_13_19.xlsx", skiprows=1, index_col=False)

In [11]:
len(canelink_dl_disb)

7788

In [12]:
# Rename key field in COD DL file to match label in CaneLink DL disbursements file
dl_booked = dl_gt_0.rename(columns={'AWARD IDENTIFIER': 'Ln App ID'})

In [13]:
len(dl_booked)

1287

In [14]:
canelink_dl_disb.head(3)

,Ln App ID,ID,Aid Yr,SUM(A.DISBURSED_BALANCE)
0,067906243U19G01536001,54298994,2019,1054
1,053905994S19G01536001,54314508,2019,1485
2,356945980S19G01536001,54318203,2019,1732


In [15]:
# Bring DOE total net amt into CL file and join by Ln App ID
dl_compare = canelink_dl_disb.merge(dl_booked[['Ln App ID','TOTAL NET AMT']],on="Ln App ID",how='left')

In [16]:
#Replace NaN values with 0 for total net amt
dl_compare = dl_compare.fillna(value={'TOTAL NET AMT':0})

In [17]:
len(dl_compare)

7788

In [18]:
dl_compare.head(3)

,Ln App ID,ID,Aid Yr,SUM(A.DISBURSED_BALANCE),TOTAL NET AMT
0,067906243U19G01536001,54298994,2019,1054,0.0
1,053905994S19G01536001,54314508,2019,1485,0.0
2,356945980S19G01536001,54318203,2019,1732,0.0


In [19]:
# Add discrepancy column (difference between CL disbursed balance and DL Total net amt)
dl_compare['DISCREPANCY'] = dl_compare['SUM(A.DISBURSED_BALANCE)'] - dl_compare['TOTAL NET AMT']

In [20]:
dl_compare.head(3)

,Ln App ID,ID,Aid Yr,SUM(A.DISBURSED_BALANCE),TOTAL NET AMT,DISCREPANCY
0,067906243U19G01536001,54298994,2019,1054,0.0,1054.0
1,053905994S19G01536001,54314508,2019,1485,0.0,1485.0
2,356945980S19G01536001,54318203,2019,1732,0.0,1732.0


In [21]:
len(dl_compare)

7788

In [22]:
# Coding for Match Status column
dl_compare.loc[dl_compare['DISCREPANCY'] == 0, 'Match Status'] = 'Booked, Matched'
dl_compare.loc[(dl_compare['DISCREPANCY'] > 0) & (dl_compare['TOTAL NET AMT'] == 0), 'Match Status'] = 'Unbooked'
dl_compare.loc[(dl_compare['DISCREPANCY'] > 0) & (dl_compare['TOTAL NET AMT'] != 0) & (dl_compare['SUM(A.DISBURSED_BALANCE)'] != 0), 'Match Status'] = 'Booked, Mismatched'
dl_compare.loc[dl_compare['DISCREPANCY'] < 0, 'Match Status'] = 'Booked, Other'

In [23]:
dl_compare.head(49)

,Ln App ID,ID,Aid Yr,SUM(A.DISBURSED_BALANCE),TOTAL NET AMT,DISCREPANCY,Match Status
0,067906243U19G01536001,54298994,2019,1054,0.0,1054.0,Unbooked
1,053905994S19G01536001,54314508,2019,1485,0.0,1485.0,Unbooked
2,356945980S19G01536001,54318203,2019,1732,0.0,1732.0,Unbooked
3,048021702S19G01536001,54290582,2019,1485,0.0,1485.0,Unbooked
4,039668904S19G01536001,54290848,2019,1485,0.0,1485.0,Unbooked
5,595876488U19G01536001,54291352,2019,0,0.0,0.0,"Booked, Matched"
6,082905356S19G01536001,54293820,2019,1485,0.0,1485.0,Unbooked
7,081907934S19G01536001,54296335,2019,1485,0.0,1485.0,Unbooked
8,219578093U19G01536001,54296551,2019,0,0.0,0.0,"Booked, Matched"
9,592933843U19G01536001,54277532,2019,0,0.0,0.0,"Booked, Matched"


In [24]:
# Coding for deriving Campus from Ln App ID
dl_compare.loc[dl_compare['Ln App ID'].str[12:18] == 'G01536', 'Campus'] = 'Gables'
dl_compare.loc[dl_compare['Ln App ID'].str[12:18] == 'E00532', 'Campus'] = 'Law'
dl_compare.loc[dl_compare['Ln App ID'].str[12:18] == 'E00533', 'Campus'] = 'Med'

In [25]:
# Coding for deriving School Code from Ln App ID
dl_compare['School Code'] = dl_compare['Ln App ID'].str[12:18]

In [26]:
# Coding for deriving Career from Ln App ID
dl_compare.loc[dl_compare['Ln App ID'].str[12:18] == 'G01536', 'Career'] = 'UGRD/GRAD'
dl_compare.loc[dl_compare['Ln App ID'].str[12:18] == 'E00532', 'Career'] = 'LAW'
dl_compare.loc[dl_compare['Ln App ID'].str[12:18] == 'E00533', 'Career'] = 'MED'

In [27]:
dl_compare.head(8)

,Ln App ID,ID,Aid Yr,SUM(A.DISBURSED_BALANCE),TOTAL NET AMT,DISCREPANCY,Match Status,Campus,School Code,Career
0,067906243U19G01536001,54298994,2019,1054,0.0,1054.0,Unbooked,Gables,G01536,UGRD/GRAD
1,053905994S19G01536001,54314508,2019,1485,0.0,1485.0,Unbooked,Gables,G01536,UGRD/GRAD
2,356945980S19G01536001,54318203,2019,1732,0.0,1732.0,Unbooked,Gables,G01536,UGRD/GRAD
3,048021702S19G01536001,54290582,2019,1485,0.0,1485.0,Unbooked,Gables,G01536,UGRD/GRAD
4,039668904S19G01536001,54290848,2019,1485,0.0,1485.0,Unbooked,Gables,G01536,UGRD/GRAD
5,595876488U19G01536001,54291352,2019,0,0.0,0.0,"Booked, Matched",Gables,G01536,UGRD/GRAD
6,082905356S19G01536001,54293820,2019,1485,0.0,1485.0,Unbooked,Gables,G01536,UGRD/GRAD
7,081907934S19G01536001,54296335,2019,1485,0.0,1485.0,Unbooked,Gables,G01536,UGRD/GRAD


In [28]:
len(dl_compare)

7788

In [29]:
# Select Unbooked and Mismatched loans only
dl_unbooked_mismatched = dl_compare.loc[(dl_compare['Match Status'] == "Unbooked") | 
                                        (dl_compare['Match Status'] == "Booked, Mismatched")]

In [30]:
len(dl_unbooked_mismatched)

4080

In [31]:
dl_unbooked_mismatched.head(8)

,Ln App ID,ID,Aid Yr,SUM(A.DISBURSED_BALANCE),TOTAL NET AMT,DISCREPANCY,Match Status,Campus,School Code,Career
0,067906243U19G01536001,54298994,2019,1054,0.0,1054.0,Unbooked,Gables,G01536,UGRD/GRAD
1,053905994S19G01536001,54314508,2019,1485,0.0,1485.0,Unbooked,Gables,G01536,UGRD/GRAD
2,356945980S19G01536001,54318203,2019,1732,0.0,1732.0,Unbooked,Gables,G01536,UGRD/GRAD
3,048021702S19G01536001,54290582,2019,1485,0.0,1485.0,Unbooked,Gables,G01536,UGRD/GRAD
4,039668904S19G01536001,54290848,2019,1485,0.0,1485.0,Unbooked,Gables,G01536,UGRD/GRAD
6,082905356S19G01536001,54293820,2019,1485,0.0,1485.0,Unbooked,Gables,G01536,UGRD/GRAD
7,081907934S19G01536001,54296335,2019,1485,0.0,1485.0,Unbooked,Gables,G01536,UGRD/GRAD
10,159805339U19G01536001,54278094,2019,1485,0.0,1485.0,Unbooked,Gables,G01536,UGRD/GRAD


In [32]:
# Select loans booked in COD DL file that have any amount not disbursed in CaneLink
dl_cod_not_in_canelink = dl_compare.loc[dl_compare['Match Status'] == "Booked, Other"]

In [33]:
len(dl_cod_not_in_canelink)

6

In [34]:
dl_cod_not_in_canelink.head(3)

,Ln App ID,ID,Aid Yr,SUM(A.DISBURSED_BALANCE),TOTAL NET AMT,DISCREPANCY,Match Status,Campus,School Code,Career
1335,028686044U19E00533001,54131908,2019,0,20035.0,-20035.0,"Booked, Other",Med,E00533,MED
2359,589610558U19E00532001,54299535,2019,0,10141.0,-10141.0,"Booked, Other",Law,E00532,LAW
2694,303025615U19E00532001,54329525,2019,0,10141.0,-10141.0,"Booked, Other",Law,E00532,LAW


In [35]:
dl_cod_not_in_canelink.pivot_table(index=["Match Status"],values=["DISCREPANCY"],
                       columns=["Campus"], aggfunc=[np.sum,len],margins=True)

sum                           len          
              DISCREPANCY                   DISCREPANCY          
Campus                Law      Med      All         Law  Med  All
Match Status                                                     
Booked, Other    -40841.0 -36794.0 -77635.0         3.0  3.0  6.0
All              -40841.0 -36794.0 -77635.0         3.0  3.0  6.0

In [36]:
len(dl_compare["TOTAL NET AMT"])  

7788

In [37]:
summary_prep = pd.DataFrame(data=dl_compare.groupby(["Campus"])["SUM(A.DISBURSED_BALANCE)","TOTAL NET AMT",
                                                                   "DISCREPANCY"].sum())

In [38]:
summary_prep

,SUM(A.DISBURSED_BALANCE),TOTAL NET AMT,DISCREPANCY
Campus,,,
Gables,19764379,0.0,19764379.0
Law,9834502,7952763.0,1881739.0
Med,12411712,11749007.0,662705.0


In [39]:
summary_prep = pd.DataFrame(data=dl_compare.groupby(["Campus"])["SUM(A.DISBURSED_BALANCE)","TOTAL NET AMT",
                                                                   "DISCREPANCY"].sum()).T

In [40]:
summary_prep["TOTAL"] = summary_prep["Gables"]+summary_prep["Law"]+summary_prep["Med"]

In [41]:
summary_prep.index.names = ['AY '+aid_year+' as of '+date[1:5]]

In [42]:
summary_prep.rename(index={'SUM(A.DISBURSED_BALANCE)': 'CaneLink DL Disbursed Amt',
                           'TOTAL NET AMT': 'COD Total Net Amt for DL Disbursed per DOE',
                    'DISCREPANCY': 'Net Discrepancy between COD and CaneLink'}, inplace=True)

In [43]:
summary_prep

Campus,Gables,Law,Med,TOTAL
AY 2018-2019 as of 8.13,,,,
CaneLink DL Disbursed Amt,19764379.0,9834502.0,12411712.0,42010593.0
COD Total Net Amt for DL Disbursed per DOE,0.0,7952763.0,11749007.0,19701770.0
Net Discrepancy between COD and CaneLink,19764379.0,1881739.0,662705.0,22308823.0


In [44]:
summary_prep2_count = pd.DataFrame(data=dl_unbooked_mismatched.groupby(["Campus"])["DISCREPANCY"].count()).T
summary_prep2_sum = pd.DataFrame(data=dl_unbooked_mismatched.groupby(["Campus"])["DISCREPANCY"].sum()).T

In [45]:
summary_prep2_count

Campus,Gables,Law,Med
DISCREPANCY,3895,135,50


In [46]:
summary_prep2_sum

Campus,Gables,Law,Med
DISCREPANCY,19764379.0,1922580.0,699499.0


In [47]:
# To control for campuses where no data exists for unbooked/mismatched loans
summary_prep2_count["Gables"] = dl_unbooked_mismatched.loc[dl_unbooked_mismatched['Campus'] == "Gables"].count()
summary_prep2_sum["Gables"] = dl_unbooked_mismatched.loc[dl_unbooked_mismatched['Campus'] == "Gables"].sum()
summary_prep2_count["Med"] = dl_unbooked_mismatched.loc[dl_unbooked_mismatched['Campus'] == "Med"].count()
summary_prep2_sum["Med"] = dl_unbooked_mismatched.loc[dl_unbooked_mismatched['Campus'] == "Med"].sum()
summary_prep2_count["Law"] = dl_unbooked_mismatched.loc[dl_unbooked_mismatched['Campus'] == "Law"].count()
summary_prep2_sum["Law"] = dl_unbooked_mismatched.loc[dl_unbooked_mismatched['Campus'] == "Law"].sum()

In [48]:
summary_prep2_count

Campus,Gables,Law,Med
DISCREPANCY,3895,135,50


In [49]:
summary_prep2_sum

Campus,Gables,Law,Med
DISCREPANCY,1.97644e+07,1.92258e+06,699499


In [50]:
summary_prep2_count["TOTAL"] = summary_prep2_count["Gables"]+summary_prep2_count["Law"]+summary_prep2_count["Med"]
summary_prep2_sum["TOTAL"] = summary_prep2_sum["Gables"]+summary_prep2_sum["Law"]+summary_prep2_sum["Med"]

In [51]:
summary_prep2_count.index.names = ['AY '+aid_year+' as of '+date[1:5]]
summary_prep2_sum.index.names = ['AY '+aid_year+' as of '+date[1:5]]

In [52]:
summary_prep2_count.rename(index={'DISCREPANCY': 'Unbooked + Mismatched Loan Count'}, inplace=True)
summary_prep2_sum.rename(index={'DISCREPANCY': 'Sum of Unbooked + Mismatched Loan Disbursements'}, inplace=True)

In [53]:
summary_prep2_count

Campus,Gables,Law,Med,TOTAL
AY 2018-2019 as of 8.13,,,,
Unbooked + Mismatched Loan Count,3895,135,50,4080


In [54]:
summary_prep2_sum

Campus,Gables,Law,Med,TOTAL
AY 2018-2019 as of 8.13,,,,
Sum of Unbooked + Mismatched Loan Disbursements,1.97644e+07,1.92258e+06,699499,2.23865e+07


In [55]:
summary_prep3_count = pd.DataFrame(data=dl_cod_not_in_canelink.groupby(["Campus"])["DISCREPANCY"].count()).T
summary_prep3_sum = pd.DataFrame(data=dl_cod_not_in_canelink.groupby(["Campus"])["DISCREPANCY"].sum()).T

In [56]:
# To control for campuses where no data exists for COD disbursed and not in CaneLink
summary_prep3_count["Gables"] = dl_cod_not_in_canelink.loc[dl_cod_not_in_canelink['Campus'] == "Gables"].count()
summary_prep3_sum["Gables"] = dl_cod_not_in_canelink.loc[dl_cod_not_in_canelink['Campus'] == "Gables"].sum()
summary_prep3_count["Med"] = dl_cod_not_in_canelink.loc[dl_cod_not_in_canelink['Campus'] == "Med"].count()
summary_prep3_sum["Med"] = dl_cod_not_in_canelink.loc[dl_cod_not_in_canelink['Campus'] == "Med"].sum()
summary_prep3_count["Law"] = dl_cod_not_in_canelink.loc[dl_cod_not_in_canelink['Campus'] == "Law"].count()
summary_prep3_sum["Law"] = dl_cod_not_in_canelink.loc[dl_cod_not_in_canelink['Campus'] == "Law"].sum()

In [57]:
summary_prep3_count["TOTAL"] = summary_prep3_count["Gables"]+summary_prep3_count["Law"]+summary_prep3_count["Med"]
summary_prep3_sum["TOTAL"] = summary_prep3_sum["Gables"]+summary_prep3_sum["Law"]+summary_prep3_sum["Med"]

In [58]:
summary_prep3_count.index.names = ['AY '+aid_year+' as of '+date[1:5]]
summary_prep3_sum.index.names = ['AY '+aid_year+' as of '+date[1:5]]

In [59]:
summary_prep3_count.rename(index={'DISCREPANCY': 'Number of loans with COD disbursements not in PS'}, inplace=True)
summary_prep3_sum.rename(index={'DISCREPANCY': 'Sum of COD disbursements not in PS'}, inplace=True)

In [60]:
summary_prep3_count

Campus,Law,Med,Gables,TOTAL
AY 2018-2019 as of 8.13,,,,
Number of loans with COD disbursements not in PS,3,3,0,6


In [61]:
summary_prep3_sum

Campus,Law,Med,Gables,TOTAL
AY 2018-2019 as of 8.13,,,,
Sum of COD disbursements not in PS,-40841,-36794,0.0,-77635


In [62]:
summary_concat = [summary_prep, summary_prep2_count, summary_prep2_sum, summary_prep3_count, summary_prep3_sum]
dl_summary = pd.concat(summary_concat)

In [63]:
dl_summary

,Gables,Law,Med,TOTAL
AY 2018-2019 as of 8.13,,,,
CaneLink DL Disbursed Amt,1.97644e+07,9.8345e+06,1.24117e+07,4.20106e+07
COD Total Net Amt for DL Disbursed per DOE,0,7.95276e+06,1.1749e+07,1.97018e+07
Net Discrepancy between COD and CaneLink,1.97644e+07,1.88174e+06,662705,2.23088e+07
Unbooked + Mismatched Loan Count,3895,135,50,4080
Sum of Unbooked + Mismatched Loan Disbursements,1.97644e+07,1.92258e+06,699499,2.23865e+07
Number of loans with COD disbursements not in PS,0,3,3,6
Sum of COD disbursements not in PS,0,-40841,-36794,-77635


In [64]:
writer = pd.ExcelWriter(path_final + '/' + aid_year + ' Direct Loan Disbursement Reconciliation - 2019.' + date + '.xlsx')
dl_summary.to_excel(writer,'Summary')
dl_unbooked_mismatched.to_excel(writer,'Unbooked+Mismatched by Ln_ID', index=False)
dl_cod_not_in_canelink.to_excel(writer,'CoD > CaneLink by Ln_ID', index=False)
dl_compare.to_excel(writer,'All Loans by Ln ID', index=False)
canelink_dl_disb.to_excel(writer,'Orig CS Query', index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Summary']
worksheet2 = writer.sheets['Unbooked+Mismatched by Ln_ID']
worksheet3 = writer.sheets['CoD > CaneLink by Ln_ID']
worksheet4 = writer.sheets['All Loans by Ln ID']
worksheet5 = writer.sheets['Orig CS Query']

# Add some cell formats.
format1 = workbook.add_format({'num_format': '$#,##0.00', 'bold': True, 'border': '1'})
format2 = workbook.add_format({'num_format': '$#,##0.00', 'border': '1'})
format3 = workbook.add_format({'bold': True, 'font_color': 'orange', 'border': '1'})
format4 = workbook.add_format({'num_format': '$#,##0.00', 'bold': True, 'font_color': 'orange', 'border': '1'})
format5 = workbook.add_format({'border': '1'})

# Note: It isn't possible to format any cells that already have a format such
# as the index or headers or any cells that contain dates or datetimes.

# Set the column width and format.
worksheet.set_column('A:A', 51)
worksheet.set_column('B:E', 15)
worksheet.set_row(1, None, format2)
worksheet.set_row(2, None, format2)
worksheet.set_row(3, None, format1)
worksheet.set_row(4, None, format3)
worksheet.set_row(5, None, format4)
worksheet.set_row(6, None, format5)
worksheet.set_row(7, None, format2)
worksheet2.set_column('A:A', 22)
worksheet2.set_column('B:C', 18)
worksheet2.set_column('D:D', 27)
worksheet2.set_column('E:J', 18)
worksheet3.set_column('A:A', 22)
worksheet3.set_column('B:C', 18)
worksheet3.set_column('D:D', 27)
worksheet3.set_column('E:J', 18)
worksheet4.set_column('A:A', 22)
worksheet4.set_column('B:C', 18)
worksheet4.set_column('D:D', 27)
worksheet4.set_column('E:J', 18)
worksheet5.set_column('A:A', 22)
worksheet5.set_column('B:C', 18)
worksheet5.set_column('D:D', 27)
worksheet5.set_column('E:J', 18)
#worksheet.write(4, 1, None, format3)
#worksheet.write(5, 1, None, format4)

# Set the format but not the column width.
# worksheet.set_column('C:C', None, format2)

writer.save()